In [1]:
import matplotlib.pylab as plt
import sys, os
import numpy as np
import pprint
import pandas as pd
from itertools import compress
%matplotlib inline

from tensorboard.backend.event_processing import event_accumulator as ea
from tensorboard.backend.event_processing import event_multiplexer as eam

from scipy.interpolate import spline
from scipy import ndimage
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:

def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)
    
def get_summaries(x, run, tag):
    outcome = [(s.step, s.value) for s in x.Scalars(run,tag)]
    outcome = np.array(outcome)
    return outcome[:,0], outcome[:,1]

def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def add_plot_sum(x, run, tag, sigma_smooth = 1, **kwargs):
    step, val = get_summaries(x, run,tag)
    plt.plot(step,val,  'r-', **kwargs)
    smooth_val = ndimage.gaussian_filter1d(val, sigma_smooth)
    #smooth_step = ndimage.gaussian_filter1d(step, sigma_smooth)
    if False: ## use tag as a label
        plt.plot(step,smooth_val,  'r-', label = tag, alpha = 1, color = kwargs['color'])
    else:
        plt.plot(step,smooth_val,  'r-', label = run, alpha = 1, color = kwargs['color'])

colors = ['blue', 'green', 'goldenrod', 'purple', 'royalblue','lime','cyan','magenta','red', ]


save_path='/home/andresro/iccv2019/tables'

lowermodelnames=['MODELcount_PATCH16_16_SCALE4_CH12_Lr0.5_Lsr1.0_Lw1.0_hrlab_sq2', #'MODELcount_PATCH16_16_SCALE4_CH12_Lr0.5_Lsr1.0_Lw1.0LOWER_hrlab_sq2',             
                 'MODELcount_PATCH16_16_SCALE2_CH12_Lr0.5_Lsr0.0_Lw1.0LOWER_sq2',
                 'MODELcount_PATCH16_16_SCALE2_CH12_Lr0.5_Lsr0.0_Lw1.0LOWER_hrlab_sq2', # 'MODELcount_PATCH16_16_SCALE2_CH12_Lr0.5_Lsr0.0_Lw1.0_hrlab_sq2after1'
                ]


## Domain Adaption

In [15]:
path='/home/pf/pfstaff/projects/andresro/sparse/training_leon/snapshots'
names = ['100','2','1','0.3']
tablefile=save_path+"/results_domain.tex"

names1 = []
sum_dirs = []
obj=['coco','palm']

model= 'MODELcountHR_ld_PATCH16_16_SCALE8_CH12_Lr0.5_Lsr0.0_Lw1.0_domainRev_sq2'
tag='Rev8d'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcountHR_lc_PATCH16_16_SCALE8_CH12_Lr0.5_Lsr0.0_Lw1.0_domainRev_sq2'
tag='Rev8c'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])


model= 'MODELcountHR_le_PATCH16_16_SCALE8_CH12_Lr0.5_Lsr0.0_Lw1.0_domainRev_sq2'
tag='Rev8e'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcountHR_ld_PATCH16_16_SCALE8_CH12_Lr0.5_Lsr0.0_Lw1.0_domainL1_sq2'
tag='L18d'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcountHR_le_PATCH16_16_SCALE8_CH12_Lr0.5_Lsr0.0_Lw1.0_domainL1_sq2'
tag='L18e'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcountHR_ld_PATCH16_16_SCALE16_CH12_Lr0.5_Lsr0.0_Lw1.0_domainRev_sq2'
tag='Rev16d'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcountHR_ld_PATCH16_16_SCALE32_CH12_Lr0.5_Lsr0.0_Lw1.0_domainRev_sq2'
tag='Rev32d'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcountHR_ld_PATCH16_16_SCALE16_CH12_Lr0.5_Lsr0.0_Lw1.0_domainL1_sq2'
tag='L116d'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcountHR_ld_PATCH16_16_SCALE32_CH12_Lr0.5_Lsr0.0_Lw1.0_domainL1_sq2'
tag='L132d'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])


tag=''
for model in lowermodelnames:
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])


index = [os.path.isdir(dir_) for dir_ in sum_dirs]

d_ = lambda x: list(dict.fromkeys(x))

names1 = list(compress(names1, index))
sum_dirs=list(compress(sum_dirs,index))

# [(x, y) for x,y in zip(names1,sum_dirs) if x in d_(names1)]

In [16]:
df =pd.DataFrame({'Name':names1}).drop_duplicates(keep=False)
new = df["Name"].str.split("_", n = 3, expand = True) 
df.set_index('Name')
df["Object"]= new[0] 
df["Area"]= new[1].astype(np.float)
df["Tag"]= new[2] 

x1 = eam.EventMultiplexer({key: value for (key, value) in zip(names1,sum_dirs) if key in d_(names1)})
del names1, sum_dirs
x1.Reload()

variables = ['metrics/iou','metrics/mae'] #, 'metrics/mse', 'metrics/recall']

for key in x1.Runs().keys():
    step = None
    for i, var_ in enumerate(variables):
        x_ = x1.Scalars(key,var_)
        step = np.argmax([i.value for i in x_]) if step is None else step
        x = x_[step]
        if not var_ in df.keys() or np.isnan(df[df.Name == key][var_]).any():
            # TODO add if key already exists
            df.loc[df.Name == key,var_] = x.value
    df.loc[df.Name == key,'step'] = x.step

df['id1'] = df['Object']+'_'+df['Tag']
d1 = df.sort_values(by=['Area','metrics/iou']).pivot(index='id1', columns='Area', values=variables)

def process_index(k):
    return tuple(k.split("_"))
d1.index = pd.MultiIndex.from_tuples([process_index(k) for k,v in d1.iterrows()])

latex_table = d1.to_latex(float_format=lambda x: '' if np.isnan(x) else '%.3f' % x)
with open(tablefile, "w") as text_file:
    text_file.write(latex_table)
    
d1

metrics/iou                               metrics/mae           \
Area              0.3       1.0       2.0       100.0       0.3      1.0     
coco                NaN  0.571977  0.601682  0.649143         NaN  0.28164   
     L116d     0.575023       NaN  0.641537  0.668828    0.286067      NaN   
     L132d     0.557991       NaN  0.652210  0.667681    0.284418      NaN   
     L18d      0.571903       NaN  0.651473  0.685584    0.277778      NaN   
     L18e      0.570081       NaN  0.652317  0.673622    0.288362      NaN   
     Rev16d         NaN       NaN       NaN  0.677067         NaN      NaN   
     Rev32d         NaN       NaN       NaN  0.672076         NaN      NaN   
     Rev8c          NaN       NaN       NaN  0.686769         NaN      NaN   
     Rev8d          NaN       NaN       NaN  0.685245         NaN      NaN   
     Rev8e          NaN       NaN       NaN  0.679048         NaN      NaN   
palm           0.646361       NaN  0.675001  0.647938    0.274230      NaN   
     L116d     0.628189       NaN  0.681759  0.653574    0.275177      NaN   
     L132d     0.628947       NaN  0.669723  0.661756    0.282403      NaN   
     L18d      0.595860       NaN  0.659868  0.653662    0.247919      NaN   
     L18e      0.587806       NaN  0.654343  0.653558    0.257154      NaN   
     Rev16d         NaN       NaN       NaN  0.649616         NaN      NaN   
     Rev32d         NaN       NaN       NaN  0.658068         NaN      NaN   
     Rev8c          NaN       NaN       NaN  0.653475         NaN      NaN   
     Rev8d          NaN       NaN       NaN  0.652726         NaN      NaN   
     Rev8e          NaN       NaN       NaN  0.649720         NaN      NaN   

                                 
Area            2.0       100.0  
coco         0.258722  0.218290  
     L116d   0.219824  0.205424  
     L132d   0.232562  0.225443  
     L18d    0.221773  0.192086  
     L18e    0.221053  0.195736  
     Rev16d       NaN  0.228493  
     Rev32d       NaN  0.210447  
     Rev8c        NaN  0.214883  
     Rev8d        NaN  0.197388  
     Rev8e        NaN  0.194542  
palm         0.225338  0.216327  
     L116d   0.240963  0.216322  
     L132d   0.247597  0.219450  
     L18d    0.245993  0.204841  
     L18e    0.246440  0.207031  
     Rev16d       NaN  0.216834  
     Rev32d       NaN  0.224628  
     Rev8c        NaN  0.202718  
     Rev8d        NaN  0.203192  
     Rev8e        NaN  0.201419

In [5]:
np.isnan(df[df.Name == 'coco_2_']['step'])

1    False
Name: step, dtype: bool

## Semi-supervised

In [6]:
path='/home/pf/pfstaff/projects/andresro/sparse/training_leon/snapshots'
tablefile=save_path+"/results_semi.tex"
names = ['100','10','2','1','0.3']
names1 = []
sum_dirs = []
obj=['coco','palm']


model= 'MODELcount_PATCH16_16_SCALE8_CH12_Lr0.5_Lsr0.0_Lw1.0_semiRev_sq2'
tag='Rev'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcount_PATCH16_16_SCALE4_CH12_Lr0.5_Lsr0.0_Lw1.0_semiRev_sq2'
tag='Rev'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcount_PATCH16_16_SCALE2_CH12_Lr0.5_Lsr0.0_Lw1.0_semiRev_sq2'
tag='Rev'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])


tag=''
for model in lowermodelnames:
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])


index = [os.path.isdir(dir_) for dir_ in sum_dirs]

d_ = lambda x: list(dict.fromkeys(x))

names1 = list(compress(names1, index))
sum_dirs=list(compress(sum_dirs,index))


In [7]:
df =pd.DataFrame({'Name':names1}).drop_duplicates(keep=False)
new = df["Name"].str.split("_", n = 3, expand = True) 
df.set_index('Name')
df["Object"]= new[0] 
df["Area"]= new[1].astype(np.float)
df["Tag"]= new[2] 

x1 = eam.EventMultiplexer({key: value for (key, value) in zip(names1,sum_dirs) if key in d_(names1)})
del names1, sum_dirs
x1.Reload()

variables = ['metrics/iou','metrics/mae'] #, 'metrics/mse', 'metrics/recall']


for key in x1.Runs().keys():
    step = None
    for i, var_ in enumerate(variables):
        x_ = x1.Scalars(key,var_)
        step = np.argmax([i.value for i in x_]) if step is None else step
        x = x_[step]
        if not var_ in df.keys() or np.isnan(df[df.Name == key][var_]).any():
            # TODO add if key already exists
            df.loc[df.Name == key,var_] = x.value
    df.loc[df.Name == key,'step'] = x.step

df['id1'] = df['Object']+'_'+df['Tag']
d1 = df.sort_values(by=['Area','metrics/iou']).pivot(index='id1', columns='Area', values=variables)

def process_index(k):
    return tuple(k.split("_"))
d1.index = pd.MultiIndex.from_tuples([process_index(k) for k,v in d1.iterrows()])

latex_table = d1.to_latex(float_format=lambda x: '' if np.isnan(x) else '%.3f' % x)
with open(tablefile, "w") as text_file:
    text_file.write(latex_table)
    
d1

metrics/iou                                         metrics/mae  \
Area           0.3       1.0       2.0       10.0      100.0       0.3     
coco             NaN  0.571977  0.601682  0.622433  0.649143         NaN   
     Rev    0.522729  0.596261  0.617954       NaN  0.668666    0.328814   
palm        0.646361       NaN  0.675001       NaN  0.647938    0.274230   
     Rev    0.608456       NaN  0.659902  0.658762  0.652199    0.296001   

                                                  
Area         1.0       2.0       10.0      100.0  
coco      0.281640  0.258722  0.231530  0.218290  
     Rev  0.253247  0.242003       NaN  0.217485  
palm           NaN  0.225338       NaN  0.216327  
     Rev       NaN  0.228697  0.219861  0.223209

## Super Resolution

In [19]:
path='/home/pf/pfstaff/projects/andresro/sparse/training_leon/snapshots'
tablefile=save_path+"/results_sr.tex"
names = ['100','10'] #,'2','1','0.3']
names1 = []
sum_dirs = []
obj=['coco','palm']

model= 'MODELcountSR_PATCH16_16_SCALE4_CH12_Lr0.5_Lsr1.0_Lw1.0_hrlab_sq2'
tag='SR4'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])

model= 'MODELcountSR_PATCH16_16_SCALE8_CH12_Lr0.5_Lsr1.0_Lw1.0_hrlab_sq2'
tag='SR8'
names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])


tag=''
for model in lowermodelnames:
    names1.extend(['{}_{}_{}'.format(i,j,tag) for i in obj for j in names])
    sum_dirs.extend([os.path.join(path,i,i+j,model,'eval') for i in obj for j in names])


index = [os.path.isdir(dir_) for dir_ in sum_dirs]

d_ = lambda x: list(dict.fromkeys(x))

names1 = list(compress(names1, index))
sum_dirs=list(compress(sum_dirs,index))


In [20]:
df =pd.DataFrame({'Name':names1}).drop_duplicates(keep=False)
new = df["Name"].str.split("_", n = 3, expand = True) 
df.set_index('Name')
df["Object"]= new[0] 
df["Area"]= new[1].astype(np.float)
df["Tag"]= new[2] 

x1 = eam.EventMultiplexer({key: value for (key, value) in zip(names1,sum_dirs) if key in d_(names1)})
x1.Reload()

variables = ['metrics/iou','metrics/mae'] #, 'metrics/mse', 'metrics/recall']

for key in x1.Runs().keys():
    step = None
    for i, var_ in enumerate(variables):
        x_ = x1.Scalars(key,var_)
        step = np.argmax([i.value for i in x_]) if step is None else step
        x = x_[step]
        df.loc[df.Name == key,var_] = x.value
    df.loc[df.Name == key,'step'] = x.step
        
  

df['id1'] = df['Object']+'_'+df['Tag']
d1 = df.sort_values(by=['Area','metrics/iou']).pivot(index='id1', columns='Area', values=variables)

def process_index(k):
    return tuple(k.split("_"))
d1.index = pd.MultiIndex.from_tuples([process_index(k) for k,v in d1.iterrows()])

latex_table = d1.to_latex(float_format=lambda x: '' if np.isnan(x) else '%.3f' % x)
with open(tablefile, "w") as text_file:
    text_file.write(latex_table)
    
d1

metrics/iou           metrics/mae          
Area           10.0      100.0       10.0      100.0
coco        0.622433  0.649143    0.231530  0.218290
     SR4         NaN  0.690382         NaN  0.228544
     SR8         NaN  0.687832         NaN  0.262276
palm             NaN  0.647938         NaN  0.216327
     SR4    0.660035  0.678021    0.432022  0.221088

In [10]:
# variable = 'metrics/iou'
# alpha_plot = 0.1
# sigma = 5
# for idx, name in enumerate(sum_names1):
#     add_plot_sum(x1, name, variable, sigma_smooth = sigma, color = colors[idx], alpha = alpha_plot)


# plt.xlabel('Iterations')
# # plt.ylim([0,1])
# # plt.xlim([0,65000])
# plt.legend(loc='center', bbox_to_anchor=(1.2, 0.5))
# plt.title('Loss')
# #filename = os.path.join('thesis/figures', 'dcgan_Genloss350mpch-over_eta-{}.png'.format(len(dict_sum1)))
# #plt.savefig(filename, bbox_inches='tight', dpi=196 )#, bbox_extra_artists=(txt_top)) #, txt_left))  # Save Image
# plt.show()

In [11]:
# sum_dirs = ['082417_1453','082417_1455','081817_1000', '081817_1002','081817_1004','082317_1426', '082317_1427','082317_1429','082317_1431']
# sum_names1 = ['$\eta=10$','$\eta=50$','$\eta=100$', '$\eta=150$','$\eta=200$','$\eta=300$','$\eta=400$','$\eta=500$','$\eta=1000$']
#model = 'DC1_SIM'
#dict_sum1 = {key: '/Volumes/UltraFit/results/{}/{}/summaries/training'.format(model,value) for (key, value) in zip(sum_names1,sum_dirs)}
# path='/scratch/andresro/leon_work/sparse/training/snapshots/palm/'
# sum_names1 = ['100','50', '2', '1.3','0.3']
# sum_dirs = ['palm{}/MODELcountHR_PATCH16_16_SCALE4_CH12_Lr0.5_Lsr1.0_Lw1.0LOWER_hrlab_sq2'.format(x) for x in sum_names1]
# #sum_dirs = ['palm100/MODELcountHR_PATCH16_16_SCALE4_CH12_Lr0.5_Lsr1.0_Lw1.0LOWER_hrlab_sq2','palm50/MODELcountHR_PATCH16_16_SCALE4_CH12_Lr0.5_Lsr1.0_Lw1.0LOWER_hrlab_sq2']

# dict_sum1 = {key: '{}{}/eval'.format(path,value) for (key, value) in zip(sum_names1,sum_dirs)}
# del sum_dirs
# print(dict_sum1)

In [12]:
# variable = 'metrics/iou'
# alpha_plot = 0.1
# sigma = 5
# for idx, name in enumerate(sum_names1):
#     add_plot_sum(x1, name, variable, sigma_smooth = sigma, color = colors[idx], alpha = alpha_plot)


# plt.xlabel('Iterations')
# # plt.ylim([0,1])
# # plt.xlim([0,65000])
# plt.legend(loc='center', bbox_to_anchor=(1.2, 0.5))
# plt.title('Loss')
# #filename = os.path.join('thesis/figures', 'dcgan_Genloss350mpch-over_eta-{}.png'.format(len(dict_sum1)))
# #plt.savefig(filename, bbox_inches='tight', dpi=196 )#, bbox_extra_artists=(txt_top)) #, txt_left))  # Save Image
# plt.show()

In [13]:
# variable = 'Loss/GradNorm'
# alpha_plot = 0.1
# sigma = 0.5
# for idx, name in enumerate(sum_names1):
#     add_plot_sum(x1, name, variable, sigma_smooth = sigma, color = colors[idx], alpha = alpha_plot)


# plt.xlabel('Iterations')
# plt.ylim([0,1000])
# plt.xlim([0,65000])
# plt.legend(loc='center', bbox_to_anchor=(1.2, 0.5))
# plt.title(variable)
# #filename = os.path.join('thesis/figures', 'dcgan_Genloss350mpch-over_eta-{}.png'.format(len(dict_sum1)))
# #plt.savefig(filename, bbox_inches='tight', dpi=196 )#, bbox_extra_artists=(txt_top)) #, txt_left))  # Save Image
# plt.show()

In [14]:
# # sum_dirs = ['082417_1453','082417_1455','081817_1000', '081817_1002','081817_1004','082317_1426', '082317_1427','082317_1429','082317_1431']
# # sum_names1 = ['$\eta=10$','$\eta=50$','$\eta=100$', '$\eta=150$','$\eta=200$','$\eta=300$','$\eta=400$','$\eta=500$','$\eta=1000$']
# #model = 'DC1_SIM'
# #dict_sum1 = {key: '/Volumes/UltraFit/results/{}/{}/summaries/training'.format(model,value) for (key, value) in zip(sum_names1,sum_dirs)}

# sum_dirs = ['180322_1039a/DsetVal']
# sum_names1 = ['DsetVal']
# dict_sum1 = {key: '/home/pf/pfstaff/projects/barry_palm/training_leon/snapshots1/{}'.format(value) for (key, value) in zip(sum_names1,sum_dirs)}
# del sum_dirs
# print(dict_sum1)